In [1]:
from urllib import request

import tensorflow as tf

from io import StringIO
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from __future__ import print_function
from keras.layers import Dense, Activation, SimpleRNN, LSTM
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer
import wordcloud as w
import matplotlib.pyplot as plot
import torch
from transformers import  BartTokenizer, TFBartForConditionalGeneration

2024-11-24 17:57:14.103846: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 17:57:14.113297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732463834.124478   21855 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732463834.127972   21855 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 17:57:14.139532: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
shelley_frank = request.urlopen('https://www.gutenberg.org/cache/epub/84/pg84.txt').read()

In [3]:
shelley_frank = shelley_frank.decode('utf-8-sig')
print(shelley_frank[:100])

The Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus
    
This ebook is for the 


In [4]:
actual_start = 'You will rejoice'
word_index = shelley_frank.find(actual_start)

In [5]:
if word_index != -1:
    shelley_frank = shelley_frank[word_index:]

In [6]:
shelley_frank = re.sub(r'—', ' ', shelley_frank.lower())
shelley_frank = shelley_frank.replace('project gutenberg', '')
shelley_frank = nltk.word_tokenize(shelley_frank)
shelley_frank = ' '.join(shelley_frank)  

# Ensure space after punctuation if it's missing
print(shelley_frank[:100])

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which yo


In [7]:
shelley_frank = re.sub(r'[^a-zA-Z.,!?; ]', '', shelley_frank)
shelley_frank = re.sub(r'([.,!?;])', r'\1 ', shelley_frank)
shelley_frank = re.sub(r'\s+([.,!?;])', r'\1', shelley_frank)
shelley_frank = re.sub(r'\s+', ' ', shelley_frank) 
print(shelley_frank[:1000])

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. i arrived here yesterday, and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking. i am already far north of london, and as i walk in the streets of petersburgh, i feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. do you understand this feeling? this breeze, which has travelled from the regions towards which i am advancing, gives me a foretaste of those icy climes. inspirited by this wind of promise, my daydreams become more fervent and vivid. i try in vain to be persuaded that the pole is the seat of frost and desolation; it ever presents itself to my imagination as the region of beauty and delight. there, margaret, the sun is for ever visible, its broad disk just skirting the horizon and diffusing a perpetual splendour. there for wi

In [8]:
with open('./shelley.txt', 'w') as output_file:
    output_file.write(shelley_frank)

with open('./shelley.txt', 'r') as f:
    text = f.read()

sentences = re.split(r'\.|\n', text)  # This will split by periods and newlines
sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

with open('./shelley_processed.txt', 'w') as file:
    for sentence in sentences:
        file.write(sentence + '\n')

# Print the number of lines
#print(len(lines))  # Make sure it prints a value greater than 1

In [9]:
from collections import Counter
import string

def count_top_words_in_file(file_path, top_n=50):
    # Open the file in read mode
    with open(file_path, 'r') as file:
        # Read the file content
        text = file.read().lower()  # convert to lowercase to count case-insensitive
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Split the text into words
    words = text.split()
    
    # Use Counter to count occurrences of each word
    word_counts = Counter(words)
    
    # Get the top N most common words
    most_common_words = word_counts.most_common(top_n)
    
    return most_common_words

# Specify the file path
file_path = 'shelley_processed.txt'

# Get the top 10 most frequent words
top_words = count_top_words_in_file(file_path, top_n=50)

# Print the top 10 most frequent words
print("Top 10 most frequent words:")
for word, count in top_words:
    print(f"'{word}': {count}")

Top 10 most frequent words:
'the': 4373
'and': 3040
'i': 2850
'of': 2757
'to': 2174
'my': 1776
'a': 1449
'in': 1186
'that': 1033
'was': 1023
'me': 867
'with': 712
'but': 691
'had': 686
'you': 645
'he': 610
'not': 587
'which': 565
'it': 559
'as': 538
'his': 535
'for': 523
'by': 485
'on': 471
'this': 444
'from': 400
'be': 376
'her': 373
'have': 371
'when': 329
'is': 328
'at': 328
'were': 308
'your': 261
'she': 256
'or': 252
'him': 222
'all': 217
'an': 216
'if': 215
'they': 212
'so': 211
'one': 208
'will': 201
'could': 198
'are': 197
'been': 190
'their': 186
'we': 183
'would': 183


In [10]:
from tokenizers import ByteLevelBPETokenizer, trainers
from transformers import PreTrainedTokenizerFast
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

bpe_tokenizer = ByteLevelBPETokenizer()
special_tokens = ["<pad>", "<s>", "</s>"]
trainer = trainers.BpeTrainer(vocab_size=20000, min_frequency=2, special_tokens=special_tokens)

%time bpe_tokenizer.train(files='./shelley_processed.txt', vocab_size=20000, min_frequency=2, special_tokens = special_tokens)

!mkdir ./transf_shelley
model_name = './transf_shelley'
bpe_tokenizer.save_model(model_name)
# Save the tokenizer using save_pretrained to create the necessary files
# Wrap the ByteLevelBPETokenizer with PreTrainedTokenizerFast
print(bpe_tokenizer.get_vocab()["<pad>"])
tokenizer = PreTrainedTokenizerFast(tokenizer_object=bpe_tokenizer)
tokenizer.pad_token = "<pad>"
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
#tokenizer.decoder_start_token_id = tokenizer.pad_token_id  # Same as pad token
# Save the tokenizer using save_pretrained to create the necessary files
tokenizer.save_pretrained(model_name)




CPU times: user 1.66 s, sys: 464 ms, total: 2.12 s
Wall time: 201 ms
mkdir: cannot create directory ‘./transf_shelley’: File exists
0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


('./transf_shelley/tokenizer_config.json',
 './transf_shelley/special_tokens_map.json',
 './transf_shelley/tokenizer.json')

In [11]:
bpe_tokenizer.get_vocab()

{'Ġempty': 6702,
 'ouble': 3877,
 'Ġabsolute': 5595,
 'Ġmockery': 3781,
 'Ġser': 2633,
 'ted': 472,
 'Ġobstructed': 7385,
 'Ġbeloved': 1476,
 'Ġagonies': 4883,
 'Ġsuperior': 2753,
 'Ġwatched': 4637,
 'Ġben': 5080,
 'Ġnecessary': 1885,
 'man': 1153,
 'utenberg': 1590,
 'Ġtaught': 3217,
 'aculous': 6802,
 'oned': 3888,
 'Ġescap': 1964,
 'Ġtravels': 4739,
 'Ġfrequ': 3152,
 'Ġarranged': 4787,
 'Ġnur': 2285,
 'Ġdoubtless': 3405,
 'omet': 848,
 'Ġstreamed': 7001,
 'Ġunderstanding': 4144,
 'Ġown': 606,
 'Ġbent': 2805,
 'Ġroared': 5388,
 'icable': 6263,
 'Ġhighly': 4726,
 'Ġhuman': 825,
 'ote': 3031,
 'Ġask': 2653,
 'Ġder': 3520,
 'Ġwater': 1797,
 'Ġtrust': 3618,
 'beaufort': 7476,
 'Ġprecipitate': 7089,
 'Ġh': 278,
 'Ġruined': 5593,
 'owed': 804,
 'omena': 6257,
 'Ġsupple': 6753,
 'Ġresembling': 7310,
 'Ġobtain': 2116,
 'Ġshowed': 3285,
 'Ġfishermen': 5925,
 'Ġexamining': 5629,
 'Ġven': 1608,
 'Ġwonder': 1018,
 'irer': 4054,
 'Ġlecture': 4235,
 'Ġwalls': 4887,
 'Ġomnipotence': 7419,
 'ardon':

In [12]:
from transformers import PretrainedConfig

# Create configuration for the tokenizer (for future use)
from transformers import T5Config
from transformers import AutoModelForSeq2SeqLM

config = T5Config(
    vocab_size=20000,                # Set vocab size based on your tokenizer
    d_model=768,                     # Embedding size (similar to n_embd in GPT2)
    num_layers=12,                   # Number of transformer layers (encoder and decoder)
    num_heads=12,                    # Number of attention heads
    max_position_embeddings=512,     # Max sequence length   
    decoder_start_token_id = tokenizer.pad_token_id,        
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id
)

#model = AutoModelForCausalLM.from_config(config)
model = AutoModelForSeq2SeqLM.from_config(config)
#model.train()
print(config)
config.save_pretrained(model_name)
# After training, save the model weights and configuration
model.save_pretrained(model_name)

T5Config {
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "max_position_embeddings": 512,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "transformers_version": "4.46.3",
  "use_cache": true,
  "vocab_size": 20000
}



In [13]:
import torch
use_gpu = -1
if torch.cuda.is_available():
    use_gpu = 0

In [14]:
from datasets import load_dataset

datasets = load_dataset("text", data_files={"train": './shelley_processed.txt'})
datasets = datasets["train"].train_test_split(test_size=0.2)
train_dataset = datasets['train']
eval_dataset = datasets['test']

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
print(datasets['train'])

Dataset({
    features: ['text'],
    num_rows: 2474
})


In [16]:
from transformers import AutoTokenizer
# Use the directory where you saved the tokenizer files
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [36]:
# Tokenizer function for multiuse
def tokenize(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=30)
#Tokenize data 
tokenized_data = datasets.map(tokenize, batched = True, num_proc = 1, remove_columns = ['text'])


Map:   0%|          | 0/2474 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

In [37]:
print(tokenized_data['train'][2])

{'input_ids': [1853, 319, 266, 819, 7486, 14, 266, 819, 5533, 14, 353, 348, 319, 434, 717, 284, 355, 2137, 516, 1171, 282, 5437, 279, 4846, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}


In [39]:
# Hyperparams
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_data.map(
    group_texts,
    batched=True,
    batch_size=128,
    num_proc=1,
)
# CHeck that decoder works okay
tokenizer.decode(lm_datasets["train"][1]["input_ids"], skip_special_tokens = True)

Map:   0%|          | 0/2474 [00:00<?, ? examples/s]

Map:   0%|          | 0/619 [00:00<?, ? examples/s]

'! how did i cling to their dear forms, as sometimes they haunted even my waking hours, and persuadethe stars shone at intervals as the clouds passed from over them; the dark pines rose before me, and every here and there a broken tree lay onour house was the house of mourningi lay for two months on the point of death; my ravings, as i afterwards heard, were frightful; i called myself the murderer of william,am i not shunned and hated by all mankind? you, my creator, would'

In [40]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from torch.utils.tensorboard import SummaryWriter

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
#writer = SummaryWriter("logs/graphs")
#writer.add_graph(model, lm_datasets["train"][0]["input_ids"])
#writer.close()

print(tokenizer.pad_token_id)
print(model.config.pad_token_id)

from transformers import Trainer, TrainingArguments

#model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-test",
    per_device_train_batch_size=6,  
    gradient_accumulation_steps=8,  
    eval_strategy = "epoch",
    num_train_epochs = 30,
    save_total_limit=2,  
    learning_rate=0.001,
    #weight_decay=0.01,
    logging_dir="./logs",
    logging_steps = 3,
)

0
0


In [41]:
print(lm_datasets["train"][0])

{'input_ids': [1221, 1317, 652, 346, 303, 266, 977, 79, 2175, 282, 6552, 4821, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 1164, 332, 387, 539, 5903, 320, 4849, 284, 387, 279, 260, 1363, 282, 2506, 14, 480, 1616, 3277, 332, 355, 5634, 14, 260, 1368, 282, 3653, 14, 353, 438, 1853, 319, 266, 819, 7486, 14, 266, 819, 5533, 14, 353, 348, 319, 434, 717, 284, 355, 2137, 516, 1171, 282, 5437, 279, 4846, 0, 0, 0, 0, 0, 0, 2968, 319, 346, 4770, 6529, 29, 279, 294, 295, 3107, 379, 320, 659, 14, 282, 1105, 5964, 6716, 14, 360, 3438, 6692, 1860, 301, 2056, 85, 1229, 321, 655, 516, 1453, 7280, 7188, 599, 262, 528, 337, 551], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

In [43]:
%time training_out = trainer.train()
trainer.save_model(model_name)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
0,7.112400,6.637514
1,6.205400,6.048180
2,5.808700,5.820285
3,5.615200,5.612189
4,5.406300,5.438526
5,5.294300,5.315340
6,5.182300,5.220170
8,4.976700,5.076432
9,5.014000,5.029024
10,4.986600,4.986720


CPU times: user 3min 56s, sys: 49.4 s, total: 4min 46s
Wall time: 4min 46s


In [56]:
%load_ext tensorboard
%tensorboard --logdir ./logs

In [44]:
# Create models from trained model
s_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
s_tokenizer = AutoTokenizer.from_pretrained(model_name)

print(s_model.config.pad_token_id)
#print(s_tokenizer.pad_token_id)
#s_tokenizer.get_vocab()

0


In [45]:
#Txt gen attempr

prompt = 'you will rejoice'

input_ids = tokenizer(prompt, return_tensors='pt').input_ids
print(input_ids)
# model generate with causal lm  = Text generation
#decoder_input_ids = tokenizer("<pad>", add_special_tokens=False, return_tensors="pt").input_ids

#print(input_ids['attention_mask'])
#e_outputs = s_model(input_ids, decoder_input_ids = decoder_input_ids, return_dict=True)
#encoded_seq = (e_outputs.encoder_last_hidden_state,)
#print(encoded_seq)
#print(e_outputs.logits)

tensor([[ 833,  459, 3919,  584]])


In [55]:
outputs = s_model.generate(
    input_ids,
    # Gen text len
    max_length=218,           
    num_return_sequences=1,
    # Sampling should be true if using top_k or top_p word estimeation if false use greedy methods
    do_sample=True,
    top_k=5,
    top_p=0.95,
    no_repeat_ngram_size=2,
    pad_token_id=s_tokenizer.pad_token_id, 
    eos_token_id=s_tokenizer.eos_token_id,
    decoder_start_token_id=s_tokenizer.pad_token_id,
    # Set temp for less random gen
    temperature=1.0
)
print(outputs)
generated_text = s_tokenizer.decode(outputs[0] , skip_special_tokens = True)
full_output = prompt + " " + generated_text
print(full_output)

tensor([[   0,    0,   75,  341,  489,  279,  319,  284,  321,   14,  262,  395,
          489,  346,  303,  266,  888,  266,  687,  888,  429,   14,  279,  266,
          937,   14,  352,  262,  341,  695,  489,  282,  266,  672,   29,  262,
          319,  266,  279,  295,  686,   14,  295,  782,  284,  266,  819,  260,
          880,  321,    0,  396,  819,  562,   29,  266,  559,    0,  261,   14,
          353,  262,  673,  437,    0,  592,  562,   14,  266, 1407,  303,  260,
          616,  282,  295,  562,  319,  260, 1297,  266,  978,  282,  387,   14,
          425,  266,  829,    0,  289,  399,  346,  320,  306,  319,   14,  337,
          266,  754,  578,  266, 1526,  282,  260,  837,  319,  535,   14,  345,
          266, 1089,  262,  457,  535,  319,  355,   14,  320,  262,  485,  346,
           14,  284,  360,  606,    0,  505,  262,  599,  262,  740,  266, 1015,
            0, 1183,  303,  321,  279,  360,  639,  282,  393,   14,  438,  319,
          346,    0,  909,  

In [146]:

greedy_search_outputs = s_model.generate(
    input_ids,
    # Gen text len
    max_length=218,           
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    #temperature=0.7,
)


generated_text = s_tokenizer.decode(greedy_search_outputs[0], skip_special_tokens=True)
greedy_output = prompt + " " + generated_text
print(greedy_output)

you will rejoice  the most of the same of my own own of a few, and i had been the cottage, but i was a littlei had taken to the world to me, i have been a manthe most great and the night, the murderer of his father, which i am the sun had not to be the old man, my fathermy father was the greatest of this time, as a thousand man and my heart, for the day, when i could not the first been to my eyesbut i saw the door of her own father and a great, that i should be a long of your father had no great was very greathe was to you are not be my friend, in the lake, to a new and then, with the ground, he had already been my mind and in a childit was in my dear not so long and he was now and to her, who had hitherto been, a very very man; and, yet i felt the idea of which, although i can not know, of its father is, or the morning


In [147]:
beam_search_outputs = s_model.generate(
    input_ids,
    # Gen text len
    max_length=218,           
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    num_beams = 5,
    temperature=0.7,
)


generated_text = s_tokenizer.decode(beam_search_outputs[0], skip_special_tokens=True)
beam_output = prompt + " " + generated_text
print(beam_output)

you will rejoice  of the most of my father, and i had been the cottage, but i have been been a man, i could not to me; but it is a fewi had taken me, which i am not be my heart of his eyes, as i saw, for ever i shall be the world, when i should be a little, that i was to you will be me to the night, in the same other; and my eyes of this was in my friend, with the murderer of your father was a thousand time, he had already one of which he was not so long of her mother; i did not know, who, my mind and the old man; it was the day, although i can not yet you are to be him, to my own eyes and then i felt me with me in a great and when he is the first saw me from the sun had no other time; my mother, yet i might be all, or to her eyes was now at the idea of a child and in this, the door, however, while i thought of
